In [9]:
from evaluate import *
from generate_data import *
import time
from focal_Loss import *
from tqdm import trange

In [2]:
        
scip_parameters = {"branching/scorefunc": "s",
                   "separating/maxrounds": 0,
                   "limits/time": 360,
                   "conflict/enable": False,
                   "presolving/maxrounds": 0,
                   "presolving/maxrestarts": 0,
                   "separating/maxroundsroot" : 0,
                   "separating/maxcutsroot": 0,
                   "separating/maxcuts": 0,
                   "propagating/maxroundsroot" : 0,
                   "lp/presolving" : False,
                  }

# generate_dataset(scip_parameters,path = "DataSetEvaluate/",nb_cons = [800,1000],nb_var = [800,1000],density = [0.1,0.2],nb_instance=20,only_problem=True)

In [3]:
def is_realisable(solu,A):
    size_var = len(solu)
    size_cons = A.shape[0] - size_var
    Aux = A[:size_var,size_cons:]
    for i,v in enumerate(solu):
        if v:
            for i_cons in np.where(Aux[i]!=0)[0][::-1]:
                Aux = np.delete(Aux, i_cons, axis=1)
    return Aux.shape[1]

In [13]:
def fix_and_optimize(var_fix_one,var_fix_zero,lp_file):
    solver = ecole.scip.Model.from_file(lp_file)
    aspyscip = solver.as_pyscipopt()
    aspyscip.setPresolve(pyscip.SCIP_PARAMSETTING.OFF)
    for i in var_fix_one:
        aspyscip.fixVar(aspyscip.getVars()[i],1)
    for i in var_fix_zero:
        aspyscip.fixVar(aspyscip.getVars()[i],0)
    aspyscip.optimize()
    return aspyscip

def gnn_plus_scip(scip_parameters,model,lp_file,with_diving = False,threshold_one=0.9,threshold_zero=10e-5):
    H,A,nb_var = getInput(scip_parameters,lp)
    y_hat,predictions = evaluate(model,H,A,nb_var,scip_parameters,with_diving)
    predictions = predictions.detach().numpy()
    var_fixe_one,var_fixe_zero = [],[]
    for i,v in enumerate(predictions):
        if v < threshold_zero:
            var_fixe_zero.append(i)
        elif v > threshold_one:
            var_fixe_one.append(i)
    print("fix %d one and %d zero"%(len(var_fixe_one),len(var_fixe_zero)))
    return fix_and_optimize(var_fixe_one,var_fixe_zero,lp_file)


In [14]:
path_set = os.listdir("DataSetEvaluateSC1500/")
dataSet = []
hist = []#list of vector [is_gnn_realisable,gap_gnn,gap_heuristique,gap_scip,temp_gnn,temp_scip]
idx=0
for filename in path_set:
    lp = "DataSetEvaluateSC1500/"+filename
    train_name = "SC1500_BCE_3MLP_50TailleH_1batch_size_"
    aux = []
    
    time_start=time.time()
    model_path = "model/"+train_name
    criterion = nn.BCELoss()
    net = VariablePredictor(24,50,3)
    net.load_state_dict(torch.load(model_path,map_location=torch.device('cpu')))
    res_her,res_gnn,A = optimize(scip_parameters,net,lp)
    time_gnn = time.time() - time_start
    
    time_start=time.time()
    aspyscip_plus = gnn_plus_scip(scip_parameters,net,lp,with_diving=False)
    res = \
        np.array(np.around(list(ast.literal_eval(aspyscip_plus.getBestSol().__repr__()).values()), 0).astype(int).tolist())
    time_plus = time.time() - time_start
    gap_plus = aspyscip_plus.getGap()
    
    
    aux.append(is_realisable(res_gnn,A))
    if aux[0] != 0:
        aux.append(1)
    else:
        for i,v in enumerate(res_gnn):
            aspyscip_plus.setSolVal(aspyscip_plus.getBestSol(),aspyscip_plus.getVars()[i],v)
        aux.append(abs(aspyscip_plus.getDualbound() - aspyscip_plus.getObjVal())/min(aspyscip_plus.getDualbound(),aspyscip_plus.getObjVal()))
    
    for i,v in enumerate(res_her):
        aspyscip_plus.setSolVal(aspyscip_plus.getBestSol(),aspyscip_plus.getVars()[i],v)
    aux.append(abs(aspyscip_plus.getDualbound() - aspyscip_plus.getObjVal())/min(aspyscip_plus.getDualbound(),aspyscip_plus.getObjVal()))
    
    time_start=time.time()
    solver = ecole.scip.Model.from_file(lp)
    aspyscip = solver.as_pyscipopt()
    aspyscip.setPresolve(pyscip.SCIP_PARAMSETTING.OFF)
    aspyscip.optimize()
    res_scip = np.array(np.around(list(ast.literal_eval(aspyscip.getBestSol().__repr__()).values()), 0).astype(int).tolist())
    time_scip = time.time() - time_start
    gap_scip = aspyscip.getGap()
    
    aux.append(gap_plus)    
    aux.append(gap_scip)
    aux.append(time_gnn)    
    aux.append(time_plus)
    aux.append(time_scip)
    hist.append(aux)
    print("Instance %d|ConsResteGnn=%f,Gap_GNN=%f,Gap_Her=%f,Gap_Plus=%f,gap_SCIP=%f \n temp_GNN=%f,temp_GNN=%f,temp_SCIP=%f"\
              %(idx,aux[0],aux[1],aux[2],aux[3],aux[4],aux[5],aux[6],aux[7]))
    idx+=1
np.mean(hist, axis=0)
hist_SC = np.rot90(hist, -1)


fix 11 one and 1438 zero
Instance 0|ConsResteGnn=0.000000,Gap_GNN=0.000000,Gap_Her=0.000000,Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=10.049304,temp_GNN=10.733631,temp_SCIP=76.996602
fix 8 one and 1435 zero
Instance 1|ConsResteGnn=5.000000,Gap_GNN=1.000000,Gap_Her=0.000000,Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=10.389971,temp_GNN=7.909227,temp_SCIP=97.546759
fix 12 one and 1444 zero
Instance 2|ConsResteGnn=0.000000,Gap_GNN=0.000000,Gap_Her=0.000000,Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=7.856540,temp_GNN=5.320199,temp_SCIP=30.688545
fix 10 one and 1306 zero
Instance 3|ConsResteGnn=0.000000,Gap_GNN=0.019231,Gap_Her=0.019231,Gap_Plus=0.000000,gap_SCIP=0.090094 
 temp_GNN=10.142384,temp_GNN=33.503343,temp_SCIP=587.471685
pressed CTRL-C 1 times (5 times for forcing termination)
fix 10 one and 1441 zero
Instance 4|ConsResteGnn=0.000000,Gap_GNN=0.000000,Gap_Her=0.000000,Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=8.803680,temp_GNN=5.436021,temp_SCIP=13.517320
fix 6 on

fix 17 one and 1295 zero
Instance 43|ConsResteGnn=0.000000,Gap_GNN=0.010870,Gap_Her=0.010870,Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=9.566782,temp_GNN=6.935084,temp_SCIP=105.577246
fix 18 one and 1338 zero
Instance 44|ConsResteGnn=0.000000,Gap_GNN=0.000000,Gap_Her=0.000000,Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=6.733719,temp_GNN=4.313321,temp_SCIP=21.835499
fix 16 one and 1276 zero
Instance 45|ConsResteGnn=2.000000,Gap_GNN=1.000000,Gap_Her=0.019231,Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=8.785351,temp_GNN=7.634523,temp_SCIP=207.829674
fix 12 one and 1310 zero
Instance 46|ConsResteGnn=5.000000,Gap_GNN=1.000000,Gap_Her=0.034483,Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=8.719967,temp_GNN=14.061025,temp_SCIP=238.838245
fix 13 one and 1326 zero
Instance 47|ConsResteGnn=0.000000,Gap_GNN=0.011628,Gap_Her=0.011628,Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=9.009019,temp_GNN=9.369354,temp_SCIP=146.974121
fix 14 one and 1309 zero
Instance 48|ConsResteGnn=0.000000,G

fix 17 one and 1297 zero
Instance 87|ConsResteGnn=0.000000,Gap_GNN=0.000000,Gap_Her=0.000000,Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=9.225723,temp_GNN=7.493205,temp_SCIP=215.438281
fix 9 one and 1304 zero
Instance 88|ConsResteGnn=3.000000,Gap_GNN=1.000000,Gap_Her=0.031250,Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=10.058505,temp_GNN=68.710671,temp_SCIP=524.096645
fix 10 one and 1441 zero
Instance 89|ConsResteGnn=0.000000,Gap_GNN=0.000000,Gap_Her=0.000000,Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=9.615965,temp_GNN=6.003906,temp_SCIP=72.387478
fix 10 one and 1354 zero
Instance 90|ConsResteGnn=0.000000,Gap_GNN=0.024096,Gap_Her=0.024096,Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=7.376961,temp_GNN=10.322260,temp_SCIP=79.282069
fix 14 one and 1328 zero
Instance 91|ConsResteGnn=5.000000,Gap_GNN=1.000000,Gap_Her=0.000000,Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=8.086836,temp_GNN=8.855512,temp_SCIP=222.299839
fix 8 one and 1436 zero
Instance 92|ConsResteGnn=0.000000,Ga

In [15]:
print("Pourcentage de solutions realisables de GNN :",len(np.where(hist_SC[0] == 0)[0])/len(hist_SC[0]))
print("Gap des solutions realisables de GNN :",(sum(hist_SC[1])-len(np.where(hist_SC[1] == 1.0)[0]))/len(np.where(hist_SC[1] != 1.0)[0]))
print("Pourcentage de solutions bonnes de GNN+Heuristique",len(np.where(hist_SC[2]  < 1)[0])/len(hist_SC[2]))
print("Gap des solutions de GNN+Heuristique:",(sum([i if i < 1 else 0 for i in hist_SC[2]])/len(np.where(hist_SC[2]  < 1)[0])))
print("Temps pour GNN:%f"%np.mean(hist_SC[5]))
print("Temps pour GNN_SCIP:%f"%np.mean(hist_SC[6]))
print("Temps pour SICP:%f"%np.mean(hist_SC[7]))
"""
SC1000_BCE_3MLP_50TailleH_1batch_size_layernorm+cat
Pourcentage de solutions realisables de GNN : 0.75
Gap des solutions realisables de GNN : 0.005164547920810669
Pourcentage de solutions bonnes de GNN+Heuristique 0.75
Gap des solutions de GNN+Heuristique: 0.005646377601438329
Temps pour GNN:2.763247
Temps pour GNN_SCIP:2.774888
Temps pour SICP:18.922448

"""
"""

Pourcentage de solutions realisables de GNN : 0.66
Gap des solutions realisables de GNN : 0.019626304688294093
Pourcentage de solutions bonnes de GNN+Heuristique 1.0
Gap des solutions de GNN+Heuristique: 0.01899509117721394
Temps pour GNN:10.582175
Temps pour GNN_SCIP:26.073124
Temps pour SICP:278.810673

"""

Pourcentage de solutions realisables de GNN : 0.66
Gap des solutions realisables de GNN : 0.019626304688294093
Pourcentage de solutions bonnes de GNN+Heuristique 1.0
Gap des solutions de GNN+Heuristique: 0.01899509117721394
Temps pour GNN:10.582175
Temps pour GNN_SCIP:26.073124
Temps pour SICP:278.810673


'\nSC1000_BCE_3MLP_50TailleH_1batch_size_layernorm+cat\nPourcentage de solutions realisables de GNN : 0.75\nGap des solutions realisables de GNN : 0.005164547920810669\nPourcentage de solutions bonnes de GNN+Heuristique 0.75\nGap des solutions de GNN+Heuristique: 0.005646377601438329\nTemps pour GNN:2.763247\nTemps pour GNN_SCIP:2.774888\nTemps pour SICP:18.922448\n\n'

In [40]:
path_set = os.listdir("DataSet_CFL/")
dataSet = []
hist = []#list of vector [is_gnn_realisable,gap_gnn,gap_heuristique,gap_scip,temp_gnn,temp_scip]
idx=0
for filename in path_set:
    if idx<100:
        lp = "DataSet_CFL/"+filename+"/problem.lp"
        train_name = "CFL30_FL_0.5Gamma_0.25Alpha_6MLP_50TailleH_1batch_size_1-10e-6_schedulerlayernorm+cat"
        model_path = "model/"+train_name
        net = VariablePredictor(24,50,6)
        net.load_state_dict(torch.load(model_path,map_location=torch.device('cpu')))
        aux = []

        time_start=time.time()
        aspyscip_plus = gnn_plus_scip(scip_parameters,net,lp,0.9,0)
        if (aspyscip_plus.getStatus() == "optimal"):
            res = \
                np.array(np.around(list(ast.literal_eval(aspyscip_plus.getBestSol().__repr__()).values()), 0).astype(int).tolist())
            time_plus = time.time() - time_start
            gap_plus = aspyscip_plus.getGap()
        else:
            time_plus = -1
            gap_plus = 1

        time_start=time.time()
        solver = ecole.scip.Model.from_file(lp)
        aspyscip = solver.as_pyscipopt()
        aspyscip.setPresolve(pyscip.SCIP_PARAMSETTING.OFF)
        aspyscip.optimize()
        res_scip = np.array(np.around(list(ast.literal_eval(aspyscip.getBestSol().__repr__()).values()), 0).astype(int).tolist())
        time_scip = time.time() - time_start
        gap_scip = aspyscip.getGap()

        aux.append(gap_plus)    
        aux.append(gap_scip) 
        aux.append(time_plus)
        aux.append(time_scip)
        hist.append(aux)
        print("Instance %d|Gap_Plus=%f,gap_SCIP=%f \n temp_GNN=%f,temp_SCIP=%f"\
                  %(idx,aux[0],aux[1],aux[2],aux[3]))
    idx+=1
np.mean(hist, axis=0)
hist_CFL = np.rot90(hist, -1)


fix 26 one and 0 zero
Instance 0|Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=0.631356,temp_SCIP=12.324170
fix 28 one and 0 zero
Instance 1|Gap_Plus=1.000000,gap_SCIP=0.000000 
 temp_GNN=-1.000000,temp_SCIP=16.740614
fix 16 one and 0 zero
Instance 2|Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=1.255213,temp_SCIP=24.024290
fix 22 one and 0 zero
Instance 3|Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=0.296172,temp_SCIP=5.324076
fix 25 one and 0 zero
Instance 4|Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=0.286872,temp_SCIP=16.820111
fix 30 one and 0 zero
Instance 5|Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=0.551279,temp_SCIP=2.148664
fix 22 one and 0 zero
Instance 6|Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=2.768505,temp_SCIP=22.707666
fix 27 one and 0 zero
Instance 7|Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=0.958927,temp_SCIP=22.399829
fix 27 one and 0 zero
Instance 8|Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=0.315382,temp_SCIP=14.311342
fix 35 one and 0 zer

fix 30 one and 0 zero
Instance 75|Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=0.634134,temp_SCIP=13.240960
fix 22 one and 0 zero
Instance 76|Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=0.303249,temp_SCIP=1.901953
fix 29 one and 0 zero
Instance 77|Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=0.430258,temp_SCIP=14.216782
fix 29 one and 0 zero
Instance 78|Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=0.442267,temp_SCIP=12.904282
fix 23 one and 0 zero
Instance 79|Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=0.367555,temp_SCIP=2.126403
fix 22 one and 0 zero
Instance 80|Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=0.303564,temp_SCIP=8.221297
fix 27 one and 0 zero
Instance 81|Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=0.646141,temp_SCIP=15.366991
fix 24 one and 0 zero
Instance 82|Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=0.233093,temp_SCIP=1.188228
fix 30 one and 0 zero
Instance 83|Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=0.290975,temp_SCIP=2.886241
fix 25 one and 

In [53]:
print("Pourcentage de solutions realisables de GNN + Scip:",len(np.where(hist_CFL[0] != 1.00)[0])/len(hist_CFL[0]))
print("Gap des solutions realisables de GNN + Scip :",(sum(hist_CFL[0])-len(np.where(hist_CFL[0] == 1.0)[0]))/len(np.where(hist_CFL[0] != 1.0)[0]))
print("Temps pour GNN + Scip:%f"%((sum(hist_CFL[2])+len(np.where(hist_CFL[0] == -1.0)[0]))/len(np.where(hist_CFL[0] != 1.0)[0])))
print("Temps pour SCIP:%f"%np.mean(hist_CFL[3]))
"""
CFL30_FL_0.5Gamma_0.25Alpha_6MLP_50TailleH_1batch_size_1-10e-6_schedulerlayernorm+cat
Pourcentage de solutions realisables de GNN + Scip: 0.74
Gap des solutions realisables de GNN + Scip : 0.0
Temps pour GNN + Scip:0.953414
Temps pour SCIP:13.879614
"""

Pourcentage de solutions realisables de GNN + Scip: 0.74
Gap des solutions realisables de GNN + Scip : 0.0
Temps pour GNN + Scip:0.953414
Temps pour SCIP:13.879614


In [46]:
path_set = os.listdir("DataSet_CFL/")
dataSet = []
hist = []#list of vector [is_gnn_realisable,gap_gnn,gap_heuristique,gap_scip,temp_gnn,temp_scip]
idx=0
for filename in path_set:
    if idx<100:
        lp = "DataSet_CFL/"+filename+"/problem.lp"
        train_name = "CFL30_BCE_6MLP_50TailleH_1batch_size_layernorm+cat+scheduler"
        model_path = "model/"+train_name
        net = VariablePredictor(24,50,6)
        net.load_state_dict(torch.load(model_path,map_location=torch.device('cpu')))
        aux = []

        time_start=time.time()
        aspyscip_plus = gnn_plus_scip(scip_parameters,net,lp,0.9,0)
        if (aspyscip_plus.getStatus() == "optimal"):
            res = \
                np.array(np.around(list(ast.literal_eval(aspyscip_plus.getBestSol().__repr__()).values()), 0).astype(int).tolist())
            time_plus = time.time() - time_start
            gap_plus = aspyscip_plus.getGap()
        else:
            time_plus = -1
            gap_plus = 1

        time_start=time.time()
        solver = ecole.scip.Model.from_file(lp)
        aspyscip = solver.as_pyscipopt()
        aspyscip.setPresolve(pyscip.SCIP_PARAMSETTING.OFF)
        aspyscip.optimize()
        res_scip = np.array(np.around(list(ast.literal_eval(aspyscip.getBestSol().__repr__()).values()), 0).astype(int).tolist())
        time_scip = time.time() - time_start
        gap_scip = aspyscip.getGap()

        aux.append(gap_plus)    
        aux.append(gap_scip) 
        aux.append(time_plus)
        aux.append(time_scip)
        hist.append(aux)
        print("Instance %d|Gap_Plus=%f,gap_SCIP=%f \n temp_GNN=%f,temp_SCIP=%f"\
                  %(idx,aux[0],aux[1],aux[2],aux[3]))
    idx+=1
np.mean(hist, axis=0)
hist_CFL = np.rot90(hist, -1)

fix 22 one and 0 zero
Instance 0|Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=0.586061,temp_SCIP=13.245099
fix 13 one and 0 zero
Instance 1|Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=0.643669,temp_SCIP=16.970366
fix 23 one and 0 zero
Instance 2|Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=0.438555,temp_SCIP=24.170418
fix 25 one and 0 zero
Instance 3|Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=0.230624,temp_SCIP=5.225989
fix 16 one and 0 zero
Instance 4|Gap_Plus=1.000000,gap_SCIP=0.000000 
 temp_GNN=-1.000000,temp_SCIP=16.858236
fix 17 one and 0 zero
Instance 5|Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=0.511254,temp_SCIP=1.939230
fix 23 one and 0 zero
Instance 6|Gap_Plus=1.000000,gap_SCIP=0.000000 
 temp_GNN=-1.000000,temp_SCIP=23.080796
fix 12 one and 0 zero
Instance 7|Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=1.659896,temp_SCIP=22.275145
fix 13 one and 0 zero
Instance 8|Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=0.342090,temp_SCIP=9.636956
fix 9 one and 0 zero

fix 10 one and 0 zero
Instance 75|Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=4.228601,temp_SCIP=12.562457
fix 35 one and 0 zero
Instance 76|Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=0.203294,temp_SCIP=1.831798
fix 14 one and 0 zero
Instance 77|Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=0.531472,temp_SCIP=13.794885
fix 18 one and 0 zero
Instance 78|Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=0.459658,temp_SCIP=12.789782
fix 11 one and 0 zero
Instance 79|Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=0.205026,temp_SCIP=2.177919
fix 21 one and 0 zero
Instance 80|Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=0.321290,temp_SCIP=8.034435
fix 19 one and 0 zero
Instance 81|Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=3.984394,temp_SCIP=15.078002
fix 14 one and 0 zero
Instance 82|Gap_Plus=0.000000,gap_SCIP=0.000000 
 temp_GNN=0.275153,temp_SCIP=1.095963
fix 25 one and 0 zero
Instance 83|Gap_Plus=1.000000,gap_SCIP=0.000000 
 temp_GNN=-1.000000,temp_SCIP=2.937905
fix 30 one and

In [51]:
print("Pourcentage de solutions realisables de GNN + Scip:",len(np.where(hist_CFL[0] != 1.00)[0])/len(hist_CFL[0]))
print("Gap des solutions realisables de GNN + Scip :",(sum(hist_CFL[0])-len(np.where(hist_CFL[0] == 1.0)[0]))/len(np.where(hist_CFL[0] != 1.0)[0]))
print("Temps pour GNN + Scip:%f"%((sum(hist_CFL[2])+len(np.where(hist_CFL[0] == -1.0)[0]))/len(np.where(hist_CFL[0] != 1.0)[0])))
print("Temps pour SCIP:%f"%np.mean(hist_CFL[3]))
"""
CFL30_BCE_6MLP_50TailleH_1batch_size_layernorm+cat+scheduler
Pourcentage de solutions realisables de GNN + Scip: 0.87
Gap des solutions realisables de GNN + Scip : 0.0
Temps pour GNN + Scip:0.940878
Temps pour SCIP:13.909654
"""

Pourcentage de solutions realisables de GNN + Scip: 0.87
Gap des solutions realisables de GNN + Scip : 0.0
Temps pour GNN + Scip:0.940878
Temps pour SCIP:13.909654


In [5]:
class VariablePredictorDiving(nn.Module):
    def __init__(self, input_size,hidden_size,nb_MLP):
        super(VariablePredictorDiving,self).__init__()
        self.GNN = GNN(input_size,hidden_size,nb_MLP)
        self.outlayer_x = simpleMLP(hidden_size*2,1,nn.Sigmoid())    
        self.outlayer_y = simpleMLP(hidden_size*2,1,nn.Sigmoid())    
        
    def forward(self,x,A):
        x = self.GNN(x,A)
        return self.outlayer_x(x),self.outlayer_y(x)

In [6]:
path_set = os.listdir("DataSet_CFL/")
dataSet = []
hist = []#list of vector [is_gnn_realisable,gap_gnn,gap_heuristique,gap_scip,temp_gnn,temp_scip]
idx=0
for filename in path_set:
    if idx<100:
        lp = "DataSet_CFL/"+filename+"/problem.lp"
        train_name = "CFL30_SL_0.75C_32Lambda_6MLP_2"
        model_path = "model/"+train_name
        net = VariablePredictorDiving(24,50,6)
        net.load_state_dict(torch.load(model_path,map_location=torch.device('cpu')))
        aux = []

        time_start=time.time()
        aspyscip_plus = gnn_plus_scip(scip_parameters,net,lp,True,0.8,0)
        if (aspyscip_plus.getStatus() == "optimal"):
            res = \
                np.array(np.around(list(ast.literal_eval(aspyscip_plus.getBestSol().__repr__()).values()), 0).astype(int).tolist())
            time_plus = time.time() - time_start
            gap_plus = aspyscip_plus.getGap()
        else:
            time_plus = -1
            gap_plus = 1

        time_start=time.time()
        solver = ecole.scip.Model.from_file(lp)
        aspyscip = solver.as_pyscipopt()
        aspyscip.setPresolve(pyscip.SCIP_PARAMSETTING.OFF)
        aspyscip.optimize()
        res_scip = np.array(np.around(list(ast.literal_eval(aspyscip.getBestSol().__repr__()).values()), 0).astype(int).tolist())
        time_scip = time.time() - time_start
        gap_scip = aspyscip.getGap()

        aux.append(gap_plus)    
        aux.append(gap_scip) 
        aux.append(time_plus)
        aux.append(time_scip)
        hist.append(aux)
        print("Instance %d|Gap_Plus=%f,gap_SCIP=%f \n temp_GNN=%f,temp_SCIP=%f"\
                  %(idx,aux[0],aux[1],aux[2],aux[3]))
    idx+=1
np.mean(hist, axis=0)
hist_CFL = np.rot90(hist, -1)
print("Pourcentage de solutions realisables de GNN + Scip:",len(np.where(hist_CFL[0] != 1.00)[0])/len(hist_CFL[0]))
print("Gap des solutions realisables de GNN + Scip :",(sum(hist_CFL[0])-len(np.where(hist_CFL[0] == 1.0)[0]))/len(np.where(hist_CFL[0] != 1.0)[0]))
print("Temps pour GNN + Scip:%f"%((sum(hist_CFL[2])+len(np.where(hist_CFL[0] == -1.0)[0]))/len(np.where(hist_CFL[0] != 1.0)[0])))
print("Temps pour SCIP:%f"%np.mean(hist_CFL[3]))

NameError: name 'VariablePredictorDiving' is not defined

In [ ]:
path_set = os.listdir("DataSet3/")
dataSet = []
hist = []#list of vector [is_gnn_realisable,gap_gnn,gap_heuristique,gap_scip,temp_gnn,temp_scip]
idx=0
for filename in path_set:
    if idx<100:
        lp = "DataSet3/"+filename+"/problem.lp"
        train_name = "SC1000_BCE_3MLP_50TailleH_1batch_size_layernorm+cat"
        aux = []
        
        time_start=time.time()
        model_path = "model/"+train_name
        criterion = nn.BCELoss()
        net = VariablePredictor(24,50,3)
        net.load_state_dict(torch.load(model_path,map_location=torch.device('cpu')))
        res_her,res_gnn,A = optimize(scip_parameters,net,lp)
        time_gnn = time.time() - time_start

        time_start=time.time()
        aspyscip_plus = gnn_plus_scip(scip_parameters,net,lp)
        res = \
            np.array(np.around(list(ast.literal_eval(aspyscip_plus.getBestSol().__repr__()).values()), 0).astype(int).tolist())
        time_plus = time.time() - time_start
        gap_plus = aspyscip_plus.getGap()


        aux.append(is_realisable(res_gnn,A))
        if aux[0] != 0:
            aux.append(1)
        else:
            for i,v in enumerate(res_gnn):
                aspyscip_plus.setSolVal(aspyscip_plus.getBestSol(),aspyscip_plus.getVars()[i],v)
            aux.append(abs(aspyscip_plus.getDualbound() - aspyscip_plus.getObjVal())/min(aspyscip_plus.getDualbound(),aspyscip_plus.getObjVal()))

        for i,v in enumerate(res_her):
            aspyscip_plus.setSolVal(aspyscip.getBestSol(),aspyscip_plus.getVars()[i],v)
        aux.append(abs(aspyscip_plus.getDualbound() - aspyscip_plus.getObjVal())/min(aspyscip_plus.getDualbound(),aspyscip_plus.getObjVal()))

        time_start=time.time()
        solver = ecole.scip.Model.from_file(lp)
        aspyscip = solver.as_pyscipopt()
        aspyscip.setPresolve(pyscip.SCIP_PARAMSETTING.OFF)
        aspyscip.optimize()
        res_scip = np.array(np.around(list(ast.literal_eval(aspyscip.getBestSol().__repr__()).values()), 0).astype(int).tolist())
        time_scip = time.time() - time_start
        gap_scip = aspyscip.getGap()

        aux.append(gap_plus)    
        aux.append(gap_scip)
        aux.append(time_gnn)    
        aux.append(time_plus)
        aux.append(time_scip)
        hist.append(aux)
        print("Instance %d|ConsResteGnn=%f,Gap_GNN=%f,Gap_Her=%f,Gap_Plus=%f,gap_SCIP=%f \n temp_GNN=%f,temp_GNN=%f,temp_SCIP=%f"\
                  %(idx,aux[0],aux[1],aux[2],aux[3],aux[4],aux[5],aux[6],aux[7]))
        idx+=1
    np.mean(hist, axis=0)
    hist_SC = np.rot90(hist, -1)

In [10]:
path_set = os.listdir("DataSet3/")
dataSet = []
hist = []#list of vector [is_gnn_realisable,gap_gnn,gap_heuristique,gap_scip,temp_gnn,temp_scip]
idx=0
for filename in path_set:
    if idx<100:
        lp = "DataSet3/"+filename+"/problem.lp"
        train_name = "SC1500_BCE_3MLP_50TailleH_1batch_size_scheduler10e-6_layernorm+cat_"
        aux = []
        
        time_start=time.time()
        solver = ecole.scip.Model.from_file(lp)
        aspyscip = solver.as_pyscipopt()
        aspyscip.setPresolve(pyscip.SCIP_PARAMSETTING.OFF)
        aspyscip.optimize()
        res_scip = np.array(np.around(list(ast.literal_eval(aspyscip.getBestSol().__repr__()).values()), 0).astype(int).tolist())
        time_scip = time.time() - time_start
        gap_scip = aspyscip.getGap()
        print(idx,"ieme instance\nscip:",time_scip,"s")
        
#         time_start=time.time()
#         model_path = "model/"+train_name
#         criterion = nn.BCELoss()
#         net = VariablePredictor(24,50,3)
#         net.load_state_dict(torch.load(model_path,map_location=torch.device('cpu')))
#         res_her,res_gnn,A = optimize(scip_parameters,net,lp)
#         time_gnn = time.time() - time_start
#         print(time_gnn)
        
        time_start=time.time()
        model_path = "model/"+train_name
        criterion = nn.BCELoss()
        net = VariablePredictor(24,50,3)
        net.load_state_dict(torch.load(model_path,map_location=torch.device('cpu')))
        aspyscip_plus = gnn_plus_scip(scip_parameters,net,lp)
        res = \
            np.array(np.around(list(ast.literal_eval(aspyscip_plus.getBestSol().__repr__()).values()), 0).astype(int).tolist())
        time_plus = time.time() - time_start
        gap_plus = aspyscip_plus.getGap()
        print("gnn+scip:",time_plus,"s,Gap:",gap_plus,",nb vars differents:",len(np.where(res_scip!=res)[0]))
#         aux.append(is_realisable(res_gnn,A))
#         if aux[0] != 0:
#             aux.append(1)
#         else:
#             for i,v in enumerate(res_gnn):
#                 aspyscip_plus.setSolVal(aspyscip_plus.getBestSol(),aspyscip_plus.getVars()[i],v)
#             aux.append(abs(aspyscip_plus.getDualbound() - aspyscip_plus.getObjVal())/min(aspyscip_plus.getDualbound(),aspyscip_plus.getObjVal()))

#         for i,v in enumerate(res_her):
#             aspyscip_plus.setSolVal(aspyscip_plus.getBestSol(),aspyscip_plus.getVars()[i],v)
#         aux.append(abs(aspyscip_plus.getDualbound() - aspyscip_plus.getObjVal())/min(aspyscip_plus.getDualbound(),aspyscip_plus.getObjVal()))
    
#         aux.append(gap_plus)    
#         aux.append(gap_scip)
#         aux.append(time_gnn)    
        aux.append(time_plus)
        aux.append(time_scip)
        hist.append(aux)
#         print("Instance %d|ConsResteGnn=%f,Gap_GNN=%f,Gap_Her=%f,Gap_Plus=%f,gap_SCIP=%f \n temp_GNN=%f,temp_GNN=%f,temp_SCIP=%f"\
#                   %(idx,aux[0],aux[1],aux[2],aux[3],aux[4],aux[5],aux[6],aux[7]))
        idx+=1
    np.mean(hist, axis=0)
hist_SC = np.rot90(hist, -1)

pressed CTRL-C 1 times (5 times for forcing termination)

scip: 179.6641788482666 s
fix 27 one and 1923 zero
gnn+scip: 6.920862197875977 s,Gap: 0.0 ,nb vars differents: 25

scip: 45.46741724014282 s
pressed CTRL-C 1 times (5 times for forcing termination)
fix 32 one and 1924 zero
gnn+scip: 6.237977027893066 s,Gap: 0.0 ,nb vars differents: 10

scip: 479.7527918815613 s
pressed CTRL-C 1 times (5 times for forcing termination)
fix 29 one and 1923 zero
gnn+scip: 8.12468409538269 s,Gap: 0.0 ,nb vars differents: 22

scip: 1042.3294429779053 s
fix 30 one and 1894 zero
gnn+scip: 6.796987295150757 s,Gap: 0.0 ,nb vars differents: 15

scip: 337.73182559013367 s
fix 30 one and 1924 zero
gnn+scip: 6.388457298278809 s,Gap: 0.0 ,nb vars differents: 17

scip: 253.98813247680664 s
fix 40 one and 1911 zero
gnn+scip: 8.099509477615356 s,Gap: 0.0 ,nb vars differents: 19

scip: 663.7945506572723 s
fix 31 one and 1914 zero
gnn+scip: 7.468985080718994 s,Gap: 0.0 ,nb vars differents: 19

scip: 503.84798407554

In [18]:
print("Temps pour GNN + Scip:%f"%np.mean(hist_SC[0]))
print("Temps pour SCIP:%f"%np.mean(hist_SC[1]))
"""
SC1500_BCE_3MLP_50TailleH_1batch_size_scheduler10e-6_layernorm+cat_
Temps pour GNN + Scip:7.645713
Temps pour SCIP:720.763742
"""

Temps pour GNN + Scip:7.645713
Temps pour SCIP:720.763742
